In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [114]:
EMBEDDING_SIZE=64
HIDDEN_SIZE=64
BATCH_SIZE=128
NUM_STEPS=8
NUM_EPOCHS_INIT_LR=4
NUM_EPOCHS_TOTAL=12
INITIAL_LR=3e-4
LR_DECAY_RATE=0.75
MAX_NORM=5

In [115]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
training_nodes = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [116]:
training_outputs = {**summary_nodes, **training_nodes}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 64 == 63):
                print('step: {0}    loss: {1}    gradient norm: {2}     correct words: {3}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['gradient_global_norm'],
                    outputs['num_correct_predictions'],
                ))
                
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('validation perplexity:', np.exp(total_loss / total_batches))
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('test perplexity:', np.exp(total_loss / total_batches))

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 64    loss: 9.203125    gradient norm: 0.129638671875     correct words: 0
step: 128    loss: 9.1953125    gradient norm: 0.116455078125     correct words: 55
step: 192    loss: 9.171875    gradient norm: 0.1461181640625     correct words: 79
step: 256    loss: 9.078125    gradient norm: 0.218017578125     correct words: 61
step: 320    loss: 8.734375    gradient norm: 0.6064453125     correct words: 71
step: 384    loss: 7.8828125    gradient norm: 0.6142578125     correct words: 53
step: 448    loss: 7.4375    gradient norm: 0.395263671875     correct words: 51
step: 512    loss: 7.046875    gradient norm: 0.280517578125     correct words: 69
step: 576    loss: 6.98828125    gradient norm: 0.22802734375     correct words: 58
step: 640    loss: 6.95703125    gradient norm: 0.259765625     correct words: 51
step: 704    loss: 6.828125    gradient norm: 0.238037109375     correct words: 66
step: 768    loss: 7.0078125    gradient norm: 0.254638671875     correct words: 40
step: 83

step: 704    loss: 6.55859375    gradient norm: 0.239013671875     correct words: 66
step: 768    loss: 6.8359375    gradient norm: 0.260009765625     correct words: 40
step: 832    loss: 6.65625    gradient norm: 0.24853515625     correct words: 45
step: 896    loss: 6.6953125    gradient norm: 0.2484130859375     correct words: 63
validation perplexity: 816.788936503
test perplexity: 742.30901965
step: 64    loss: 6.5546875    gradient norm: 0.2406005859375     correct words: 63
step: 128    loss: 6.8046875    gradient norm: 0.23974609375     correct words: 55
step: 192    loss: 6.6171875    gradient norm: 0.266357421875     correct words: 79
step: 256    loss: 6.703125    gradient norm: 0.222412109375     correct words: 61
step: 320    loss: 6.703125    gradient norm: 0.2646484375     correct words: 71
step: 384    loss: 6.7421875    gradient norm: 0.24951171875     correct words: 53
step: 448    loss: 6.80078125    gradient norm: 0.226806640625     correct words: 51
step: 512    lo

KeyboardInterrupt: 